In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
from osgeo import gdal

/Users/hongyuwang/code/HLandscape/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
lon, lat = 117.768023,41.970009

In [2]:
lat, lon = 10.577186,103.557795

In [3]:
m = leafmap.Map(draw_control=True, layers_control=True, measure_control=True, fullscreen_control=False, attribution_control=True)
m.add_basemap("SATELLITE")
m.set_center(lon,lat,18)
m.add_marker(location=[lat, lon], draggable=False)
m

Map(center=[10.577186, 103.557795], controls=(AttributionControl(options=['position', 'prefix'], position='bot…

In [ ]:
m.user_roi_bounds

In [4]:
def download_geotiff(Name,Zoom,geotiff_path,source):
    tms_to_geotiff(output=geotiff_path,bbox=m.user_roi_bounds(), zoom=Zoom,source=source,overwrite=True)
def cal_epsg():
    center = round(m.center[1],0)
    epsg = int(4548 - round((117-center)/3,0))
    print(f'本项目中央子午线精度为：{center}，投影坐标系应为 EPSG:{epsg}')
    return epsg
def reproject_geotiff(src_filename, dst_filename, dst_srs):
    # Open the source file
    src_ds = gdal.Open(src_filename)
    if src_ds is None:
        print('Could not open source file', src_filename)
        return
    
    # Fetch the source SRS
    src_srs = src_ds.GetProjectionRef()
    
    # Create the destination file
    dst_ds = gdal.Warp(dst_filename, src_ds, dstSRS=dst_srs)
    print(src_filename,dst_filename,dst_srs)

    # Close datasets
    if dst_ds:
        dst_ds = None
    if src_ds:
        src_ds = None

def generate_tfw(geotiff_path, tfw_path):
    """Generate a TFW file from a GeoTIFF file.

    Args:
        geotiff_path: The path to the GeoTIFF file.
        tfw_path: The path to the TFW file.
    """

    dataset = gdal.Open(Re_geotiff_path)

    if dataset is None:
        raise Exception("Could not open GeoTIFF file.")

    transform = dataset.GetGeoTransform()
    transform = [transform[1], transform[2], transform[4], transform[5], transform[0], transform[3]]

    with open(tfw_path, "w") as f:
        f.write("\n".join([f"{x:.16f} " for x in transform]))

In [5]:
if __name__ == "__main__":
    image_folder = r'D:\Documents\WeChat Files\wxid_3oemgw81j4rc22\FileStorage\File\2023-10\IMG_20231030_162539(1)'
    epsg = cal_epsg()
    Name = "码头项目"
    source = "SATELLITE"
    # temp  = "D:\Temp\Satellite"
    # os.chdir(temp)
    # base_path = r"W:\path\to\image\folder"
    # os.chdir(base_path)
    os.makedirs(f"./{Name}", exist_ok=True)
    Zoom = 18
    geotiff_path = f"./{Name}/{Name}_{source}_4326.tif"
    Re_geotiff_path = f"./{Name}/{Name}_{source}_{epsg}.tif"
    download_geotiff(Name,Zoom,geotiff_path,source)
    reproject_geotiff(geotiff_path, Re_geotiff_path, f'EPSG:{epsg}')
    tfw_path = f"./{Name}/{Name}_{source}_{epsg}.tfw"
    generate_tfw(geotiff_path, tfw_path)
    os.system('start .')

本项目中央子午线精度为：104.0，投影坐标系应为 EPSG:4544
Downloaded image 001/384
Downloaded image 002/384
Downloaded image 003/384
Downloaded image 004/384
Downloaded image 005/384
Downloaded image 006/384
Downloaded image 007/384
Downloaded image 008/384
Downloaded image 009/384
Downloaded image 010/384
Downloaded image 011/384
Downloaded image 012/384
Downloaded image 013/384
Downloaded image 014/384
Downloaded image 015/384
Downloaded image 016/384
Downloaded image 017/384
Downloaded image 018/384
Downloaded image 019/384
Downloaded image 020/384
Downloaded image 021/384
Downloaded image 022/384
Downloaded image 023/384
Downloaded image 024/384
Downloaded image 025/384
Downloaded image 026/384
Downloaded image 027/384
Downloaded image 028/384
Downloaded image 029/384
Downloaded image 030/384
Downloaded image 031/384
Downloaded image 032/384
Downloaded image 033/384
Downloaded image 034/384
Downloaded image 035/384
Downloaded image 036/384
Downloaded image 037/384
Downloaded image 038/384
Downloaded ima

sh: start: command not found
